In [1]:
import os
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)

from dotenv import load_dotenv
from os import environ
load_dotenv()

True

### Set Up Api Key


In [2]:
OPENAI_API_KEY = environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Load Data Files

In [3]:
filename_fn = lambda filename: {"file_name": filename}
# automatically sets the metadata of each document according to filename_fn
documents = SimpleDirectoryReader(
    input_files=["./data/01.pdf","./data/02.pdf","./data/03.pdf","./data/04.pdf","./data/05.pdf"], file_metadata=filename_fn
).load_data()

In [5]:
print(f"Loaded {len(documents)} docs")

Loaded 91 docs


### Save Indexing

In [6]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
  # rebuild storage context
  storage_context = StorageContext.from_defaults(persist_dir="storage")
  # load index
  index = load_index_from_storage(storage_context=storage_context)

### Retrieval

In [7]:
query_engine = index.as_query_engine(response_mode="tree_summarize", similarity_top_k=5)

In [8]:
response = query_engine.query("how strong must the spinal column be?")

In [9]:
response.response

'The spinal column must be strong enough to support the trunk and extremities, protect the spinal cord and cauda equina, anchor muscles, and allow for movement of the head and trunk in multiple directions.'

In [10]:
#explore which nodes were selected for the answer
for node_with_score in response.source_nodes:
    print(node_with_score.node)

Node ID: 1171a1e8-d743-4afa-a6d8-05cd55bc36aa
Text: Anatomy and Examination of the Spine Michael W. Devereaux, MDa,b
aDepartment of Neurology, University Hospitals of Cleveland/Case
Western Reserve University, 11100 Euclid Avenue, Cleveland, OH 44106,
USA bUniversity Hospitals Health System, Richmond Heights Hospital
27100 Chardon Road, Richmond Heights, OH 44143-1116, USA A review of
the anatomy...
Node ID: fa377596-72fe-478f-bd0e-e5110f628164
Text: assess whether the spine is balanced. A spine is said to be
balanced clinically when the head is centred over the pubicsymphysis
in the coronal plane and centred over the femoralhead in the sagittal
plane ( Figure 1 ). In the coronal plane a weighted string or plumb
line can be used to further assess bal-ance. The string should be
pressed agains...
Node ID: 936d20ac-9c3c-433b-a4f1-b47b9ba51cd9
Text: The spinal canal itself is formed posterolaterally by the
laminae and lig- amentum ﬂavum, anterolaterally by the pedicles, and
anteriorly by t

In [11]:
#explore which files were selected for the answer
for node_with_score in response.source_nodes:
    print(node_with_score.node.metadata)

{'page_label': '331', 'file_name': 'data\\02.pdf'}
{'page_label': '351', 'file_name': 'data\\03.pdf'}
{'page_label': '332', 'file_name': 'data\\02.pdf'}
{'page_label': '350', 'file_name': 'data\\03.pdf'}
{'page_label': '468', 'file_name': 'data\\04.pdf'}


### Chat Completions With OpenAI

In [6]:
text = """
EXAM: MR LUMBAR SPINE WITH AND WITHOUT CONTRAST

CLINICAL INDICATION: Low back pain.

TECHNIQUE: Multiplanar multisequence MRI of the lumbar spine performed without and with intravenous
contrast.

CONTRAST: 15 mL of Clariscan

COMPARISON: None.

FINDINGS:
Assuming normal segmentation, there are 5 lumbar type vertebra with S1 completely incorporated into the
sacrum.

Postsurgical changes of L2-L4 laminectomies with posterior instrumented fusion at L2-L5. Susceptibility
artifact related to fusion hardware limits evaluation of adjacent structures.

Vertebral Bodies: Chronic-appearing mild anterior wedging of the T12 vertebral body. Vertebral body
heights are otherwise maintained. No suspicious marrow signal replacement.
Alignment: No acute lumbar subluxation. Stepwise grade 1 retrolisthesis of L1 on L2 and L2 on L3. Mild
lumbar dextrocurvature.

Extradural: There are no abnormal extradural fluid collections or masses. No foci of abnormal
enhancement.
Spinal Cord: The distal spinal cord appears normal.
Conus: The conus terminates at approximately L1

T12-L1: No spinal canal stenosis. Mild bilateral neuroforaminal stenosis due to foraminal extension of
disc bulge and facet arthropathy.

L1-L2: Severe spinal canal stenosis due to disc bulge, facet arthropathy and ligamentum flavum
redundancy. Moderate right and severe left neuroforaminal stenosis due to foraminal extension of disc
bulge and facet arthropathy.

L2-L3: No spinal canal stenosis. Mild bilateral neuroforaminal stenosis due to foraminal extension of
disc bulge and facet arthropathy.

L3-L4: No spinal canal stenosis. Mild right and moderate left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

L4-L5: No spinal canal stenosis. Mild right and moderate left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

L5-S1: No spinal canal stenosis. Moderate right and severe left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

Paraspinal Soft Tissues: Unremarkable.
Regional Soft Tissues: Simple-appearing right renal cyst measuring 6.7 x 5.8 cm.


IMPRESSION:

Postsurgical and multilevel degenerative spondylotic changes of the lumbar spine with up to severe spinal
canal and severe neuroforaminal stenosis, further discussed in level by level detail above.
"""

In [7]:
query = f"""
Assume you are a patient with limited medical knowledge who has received a medical report filled with complex terminology. You are seeking a clearer understanding of this report in two parts:

1. **Report Explanation**: First, break down the medical report, keeping the original terms but explaining their significance. Detail what each finding or measurement within the report indicates about your health. Include any abnormalities or conditions detected, explaining what each part of the scan or test represents. 

2. **Simplified Explanation**: Next, provide a simplified explanation of the report's findings as if explaining to a complete layperson or as though you were explaining it to a two-year-old. This should include:
- A plain English summary of any conditions or abnormalities found.
- Insights into how these findings relate to your overall health.
- Suggestions for potential treatment options or further diagnostic tests, based ONLY on the report's findings.
- Clarification of any complex terms or concepts in very simple language, avoiding medical jargon.

Please ensure that while simplifying, you do not omit essential medical terms; rather, introduce them with their explanations to ensure the patient fully understands their report.

Medical report: {text}
"""

In [8]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system", 
            "content": "You are a knowledgeable agent specializing in the medical domain, proficient in interpreting and analyzing medical reports with precision and expertise."
         },
        {
            "role": "user", 
            "content": query
         },
    ],
)

In [9]:
response.choices[0].message.content

'1. **Report Explanation**:\n   - **Clinical Indication**: The reason for the test was low back pain.\n   - **Technique**: MRI scan of the lower back was done from different angles both with and without contrast dye.\n   - **Contrast**: A contrast agent called Clariscan was used during part of the scan.\n   - **Findings**:\n     - **Vertebrae**: There are 5 vertebrae in the lower back, with the last one (S1) partially fused with the pelvis bone (sacrum).\n     - **Surgical Changes**: Previous surgeries at levels L2-L4 with fusion of bones using hardware, making it challenging to see nearby structures clearly.\n     - **Vertebral Bodies**: T12 vertebra shows slight front compression without major issues in height or bone marrow.\n     - **Alignment**: No sudden shifts in the spine but slight slipping of upper vertebral bones over lower ones and a mild curve to the right.\n     - **Extradural**: No unusual fluid collections or masses around the spinal cord.\n     - **Spinal Cord**: The l